In [ ]:
import requests
import re
from datetime import date
from datetime import datetime, timedelta
import pymongo

# Date & Time
created_on = date.today().strftime("%Y-%m-%d")
current_time = datetime.now().strftime("%H_%M_%S")

# Mongo db connect
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
dashboard = mongo_client["dashboard"]
cl_data = dashboard['data_collect']

# intialize some value
publish_source = 'cnn.com'
country = 'United states'
language = 'English'


headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }

url = 'https://edition.cnn.com/services/rss/'
page = requests.get(url, headers = headers, timeout = 60).text

for i in page.split('title="CNN')[1:]:
    urls = re.search('href="(.*?)"',i).group(1)
    print(urls)
    content_url = requests.get(urls, headers = headers, timeout = 60).text
    for j in content_url.split('<item>')[1:]:
        
        # source link
        links = re.search('<link>(.*?)</link>',j,re.S).group(1)
        print(links)
        source_link = requests.get(links).text
        
        
        # source headline
        headline = re.search('<title>(.*?)</title>',j,re.S)
        if headline: 
            headline = headline.group(1)
        headline = re.sub('<!\[CDATA\[|\]\]>','',str(headline),re.S)
        
        # source date
        date_str = re.search('<pubDate>\w+,(.*?)</pubDate>',j,re.S)
        if date_str:
            date_str = date_str.group(1)
        if not date_str:
            print(f"Skipping due to date :  {links}")
            continue
        date_str = re.sub(' |GMT|:','',str(date_str),re.S)
        date_obj = datetime.strptime(date_str, '%d%b%Y%H%M%S')
        ist_date_time = date_obj - timedelta(hours = 2,minutes = 30) 
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
        print(pub_date)
        print(publish_time)

        # source content
        source_content = re.search('class="el-editorial-source">(.*?)<p class="zn-body__paragraph\s*zn-body__footer"', source_link,re.S)
        if not source_content:
            source_content = re.search('property="og:description">.*?"(.*?)"', source_link,re.S)
        if not source_content:
            source_content = re.search('class="Paragraph__cite">(.*?)<div\s*class="Article__contentTail">',source_link,re.S)
        if not source_content:
            source_content = re.search('<div\s*class="Paragraph__component.*?"(.*?)<div\s*class="BasicArticle__railTop\s*obRightRail">',source_link,re.S)
        if source_content:
            source_content = source_content.group(1)
        if not source_content:
            print(f"Skipping due to content : {links}")
        
        # stripping content 
        source_content = re.sub(r'<!--.*?-->','', source_content, flags = re.S)
        source_content = re.sub('<script.*?>.*?</script>','', source_content, flags = re.S)
        source_content = re.sub('<style.*?>.*?</style>','', source_content, flags = re.S)
        source_content = re.sub('<.*?>|&nbsp;', "", source_content, flags = re.S)
        source_content= re.sub(r'\n\s*\n', '\n'*2, source_content).strip()
        print(source_content)
        
        
        # Journalist
        journalist = ''
        if not journalist: journalist = 'NA'
            
        # storing the above data in a dictionary
        clientdata ={
                        
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "publish_source": publish_source,
                        "journalist":journalist,
                        "source_headline":headline,
                        "source_content":source_content,
                        "language":language,
                        "source_link":links,
                        "country":country,
                        "source":publish_source,
                        "created_on":created_on,
                    } 

        cl_data.insert_one(clientdata)  

        
            

http://rss.cnn.com/rss/edition.rss
https://www.cnn.com/collections/intl-prince-philip-death-0409/
2021-04-09
09:59:37
Error
https://www.cnn.com/2021/01/12/world/gallery/prince-philip/index.html
2021-04-09
09:24:58
See photos of Prince Philip, the late husband of Britain&#39;s Queen Elizabeth II.
https://www.cnn.com/videos/world/2021/04/09/prince-philip-life-family-controversy-lon-orig-mkd.cnn
2021-04-09
09:06:08
Known more for his controversial comments and being one step behind the Queen, Prince Philip played an important role in the royal household. One that may have helped keep it together. CNN takes a look at his life.
https://www.cnn.com/2021/04/08/asia/myanmar-zaw-min-tun-interview-intl-hnk/index.html
2021-04-08
18:19:14
Naypyidaw, Myanmar  (CNN)"This is not a coup," said Maj. Gen. Zaw Min Tun from a gilded hall in Myanmar's purpose-built capital Naypyidaw, the city where his comrades recently ousted an elected government, detained the country's leadership, and installed a milita

2021-04-08
18:04:35
The Myanmar military has granted CNN access to the country two months after the military overthrew the democratically-elected government in a coup and has had violent clashes with protestors.
https://www.cnn.com/2021/04/09/americas/brazil-race-politics-change-intl-latam/index.html
2021-04-09
05:44:09
(CNN)Last year, more than 43,400 politicians in Brazil declared themselves to be a different race than in previous election years.Since 2014, Brazil has required electoral candidates to declare their "color/race," picking from among "Black," "White," "Brown," "Yellow," and "Indigenous." A CNN analysis of data published by the Electoral Justice, which organizes elections, shows that one in four candidates who ran for office in a previous election declared a different "color/race" in 2020 from previous such filings.The changes in politicians' self-identification reflect Brazil's rich diversity and mirror the country's evolving view on how to recognize and define demograph

2021-04-09
06:48:31
New DelhiIndia's Supreme Court on Thursday rejected a plea to stop the government from deporting to Myanmar some 150 Rohingya Muslims police detained last month, paving the way for them to be sent to a country where hundreds have been killed following a military coup.Prime Minister Narendra Modi's government has been trying to send back Rohingyas, a Muslim minority from Myanmar who have found refuge in India after fleeing persecution and waves of violence over the years.Two refugees petitioned the Supreme Court for the release of Rohingya refugees detained in the northern Jammu region last month, and to block the government from deporting them. Their plea argued that refugees in Jammu "have been illegally detained and jailed in a sub-jail now converted into a holding center."India begins deporting more than 150 Rohingya refugees to MyanmarTheir petition added the Indian constitution -- which states that no person shall be deprived of life or personal liberty -- by p

2021-04-08
21:42:33
John Boehner, former Republican House Speaker, reflects on his tenure in office in his new book, &quot;On the House: A Washington Memoir.&quot; CNN&#39;s Brian Todd breaks down some key stories from the book, including the time Alaska Rep. Don Young held a knife to his throat.
https://www.cnn.com/2021/04/09/asia/japan-lung-transplant-covid-intl-hnk/index.html
2021-04-09
05:36:57
(CNN)A Japanese woman whose lungs were severely damaged by Covid-19 has received what doctors say is the world's first lung transplant from living donors to a recovered coronavirus patient. Kyoto University Hospital said the woman underwent an 11-hour operation by a 30-strong medical team on Wednesday to transplant lung tissue from her husband and son.X-ray images show the patient's chest before the transplant operation, left, and after, right. The dark areas indicate where lung tissue has been transplanted. Covid-19 is known to cause severe lung damage in some patients, and people around th

2021-04-09
01:38:59
(CNN)Joe Manchin says it's no fun being the most powerful man in Washington.The West Virginia Democrat, who sits at the fulcrum of the Senate balance of power, suggested the US Capitol insurrection was a warning for him -- of the divisions tearing America apart and the dangers wrought by politicians who abuse their power."Something told me, 'Wait a minute. Pause. Hit the pause button.' Something's wrong. You can't have this many people split to where they want to go to war with each other," Manchin told CNN's Lauren Fox in an interview on Thursday.It might surprise some of his fellow Democrats, however, to learn that Manchin now appears to believe that the appropriate response to those horrific events is to prevent either side in the Senate from exerting their will over the other. If that means thwarting a bid to overturn supermajority filibuster rules in the Senate that could buckle Democratic dreams of a radical and historic Joe Biden presidency, that appears to b

...

2019-12-11
09:57:01
Copenhagen (CNN) — In Copenhagen, traffic is usually caused by the two-wheel variety of transportation: the bicycle. Ever since bicycles were imported into Denmark from France in 1869, they have become the main form of transportation in the Scandinavian city. In the 1920s, it wasn't unusual to see both the working class as well as high society pedaling through the streets. But with the opening of the Metro's new M3 Cityring line, commuters have a new way of getting around. While Copenhagen's Metro has always been rather efficient, many neighborhoods lacked stations, and thus, accessibility. In addition, a growing population of about 10,000 new residents per year is taxing the buses and trains already in circulation. In a city of 650,000 people, about 200,000 commute on the Metro daily, sometimes in conjunction with a bike during their journey. Related contentThese are predicted to be the safest countries in the world to visit in 2020 Greater accessibilityThe circula

2019-12-20
09:16:53
(CNN) — Budapest's craft cocktail movement has seen new bars and clubs spread across town like wildfire in recent years, making it that much harder to choose the best of the bunch.While the Hungarian capital's ruin bars offer a unique experience, the luxury bar scene here is second to none.Whether you're looking to sample handcrafted cocktails or the finest Tokaji wines, there are a multitude of glamorous bars to choose from, complete with sharply dressed staff and/or top-notch mixologists.From the High Note Skybar to Blue Fox The Bar at Kempinski, here's a run down of the finest luxury bars in Budapest.KOLLÁZS -- Brasserie &amp; Bar Kollázs Brasserie &amp; Bar is located on the ground floor of the Four Seasons Hotel Gresham Palace Budapest.Courtesy Kempinski Hotel Corvinus BudapestSet at the Four Seasons Gresham Palace, KOLLÁZS -- Brasserie &amp; Bar offers what's arguably the grandest setting in the entire city. Serving everything from Tokaji wines to bespoke cock

2019-12-02
08:38:37
(CNN) — While Budapest may be primarily known for its architecture, geothermal springs and communist heritage, the city's gastronomy scene has been gaining considerable travel attention in recent years.New and exciting fine dining spots are constantly opening up in the Hungarian capital, many headed up by prolific chefs keen to inject imagination and prestige into the Budapest dining experience.Two restaurants in the Hungarian capital were awarded new Michelin stars in the past 12 months alone, bringing the total of Michelin-awarded establishments in Budapest to six.Hungary only received its first Michelin star nine years ago, so this is a pretty remarkable turnaround.There's little doubt a culinary revolution is underway in the "Pearl of the Danube," but what's brought about this burgeoning movement?Record numbers of tourists and a booming economy have definitely played a part.With Hungary's difficult past well documented, it's fair to say that fine dining hasn't n

2019-09-02
11:53:18
(CNN) — From traditional souks, mosques and UNESCO World Heritage sites, to futuristic skyscrapers, Baku, capital of Azerbaijan, offers an eclectic mix of old and new. Among the most startling additions to the modern skyline are the Flame Towers, a trio of sky-high structures that resemble tongues of fire reaching high over the Caspian Sea. Designed by US architecture firm HOK, and with Tower 1 standing at 182 meters, they're the tallest buildings in Azerbaijan.And visitors can stay in them -- one building's residential, one is for office use, while the third, the 165-meter-tall Fairmont Baku, is a contemporary five-star hotel. Related contentThe mud volcano capital of the world Enter the flameThe towers can be seen from almost anywhere in the city, but night is when they truly come alive. Orange, red and pink LED lights dance across the towers like burning flames. For some in Baku, they're now a symbol of home."You can see Flame Towers as you fly into the airport, 

2018-10-31
10:59:54
(CNN) — "This fire has burned 4,000 years and never stopped," says Aliyeva Rahila. "Even the rain coming here, snow, wind -- it never stops burning."Ahead, tall flames dance restlessly across a 10-meter stretch of hillside, making a hot day even hotter. This is Yanar Dag -- meaning "burning mountainside" -- on Azerbaijan's Absheron Peninsula, where Rahila works as a tour guide.A side effect of the country's plentiful natural gas reserves, which sometimes leak to the surface, Yanar Dag is one of several spontaneously occurring fires to have fascinated and frightened travelers to Azerbaijan over the millennia. Venetian explorer Marco Polo wrote of the mysterious phenomena when he passed through the country in the 13th century. Other Silk Road merchants brought news of the flames as they would travel to other lands. It's why the country earned the moniker the "land of fire." Related contentThe mud volcano capital of the world Ancient religion Such fires were once plent